In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Иницијализација на Web Driver за Selenium
driver = webdriver.Chrome()
driver.get("https://kosuli.mk")
wait = WebDriverWait(driver, 10)

# Дефинирање на речник, dictionary на категориите кои ги скрејпам со големи букви
menu_targets = {
    "Брендови": {"CASA MODA", "FF-FASHION", "VENTI"},
    "Кратки ракави": {"МОДЕРН ФИТ"},
}

# Празно dictionary за чување на УРЛ's
extracted_urls = {}

for top_menu, targets in menu_targets.items():
    # Се чека елементот да биде пронајден со hover 
    link = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//a[contains(normalize-space(), '{top_menu}')]")
    ))
    actions = ActionChains(driver)
    actions.move_to_element(link).perform()

    # Се чеза за елементот да биде видлив за да може да се лоцира
    submenu = wait.until(EC.presence_of_element_located(
        (By.XPATH, f"//li[a[contains(normalize-space(), '{top_menu}')]]/ul")
    ))

    # Се зачувуваат сите линкови од секоја категорија 
    submenu_anchors = submenu.find_elements(By.TAG_NAME, "a")

    for a in submenu_anchors:
        text = a.text.strip().upper()
        href = a.get_attribute("href")
        if text in targets and href:
            # Се зачувува во варијабла линкот 
            key = text.lower().replace(" ", "_")
            extracted_urls[key] = href

# Затварање на бровсерот односно исклучување на селениум
driver.quit()

# Зачувување на варијабли, главната варијбала страницата и потоа секое урл посебно
base_url = "https://kosuli.mk"
url_casa_moda = extracted_urls.get("casa_moda")
url_ff_fashion = extracted_urls.get("ff-fashion")
url_venti = extracted_urls.get("venti")
url_modern_fit = extracted_urls.get("модерн_фит")

# Принтање на секој линк
print("Casa Moda :", url_casa_moda)
print("FF-Fashion:", url_ff_fashion)
print("Venti     :", url_venti)
print("Modern Fit:", url_modern_fit)


Casa Moda : https://kosuli.mk/product-category/casa-moda/
FF-Fashion: https://kosuli.mk/product-category/f-fashion/
Venti     : https://kosuli.mk/product-category/venti/
Modern Fit: https://kosuli.mk/product-category/modern-fit/


In [5]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

# Сетирање на селениум веб драјверот и исклучување на сликите за да биде побрзо да не се лоадираат сликите
options = Options()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

product_urls = []
page = 1

# Изминување на сите страни за да се најдат сите продукти и нивните урла
while True:
    driver.get(f"{url_ff_fashion}page/{page}/")
    time.sleep(2)

    links = driver.find_elements(By.CSS_SELECTOR, "ul.products li.product a.woocommerce-LoopProduct-link")
    if not links:
        break

    for a in links:
        href = a.get_attribute("href")
        if href and href not in product_urls:
            product_urls.append(href)

    try:
        driver.find_element(By.CSS_SELECTOR, "a.next.page-numbers")
        page += 1
    except NoSuchElementException:
        break

print(f"→ Found {len(product_urls)} products across {page} pages.")

# Скрејпање на сите продукти и нивно зачувување
data = []
for idx, href in enumerate(product_urls, start=1):
    driver.get(href)
    time.sleep(1)

    # Скрејпање на името
    name = driver.find_element(By.TAG_NAME, "h1").text.strip()

    # Скрејпање на цената
    try:
        price = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
    except NoSuchElementException:
        price = ""

    # Скрејпање на урлто на сликите
    try:
        img = driver.find_element(By.CSS_SELECTOR, "div.woocommerce-product-gallery__image img")
        image_url = img.get_attribute("data-src") or img.get_attribute("src")
    except NoSuchElementException:
        image_url = ""

    # Скрејпање на достапни величини
    sizes = []
    try:
        descr = driver.find_element(By.ID, "tab-description")
        driver.execute_script("arguments[0].scrollIntoView(true);", descr)
        div = descr.find_element(By.XPATH, ".//div[contains(text(),'Достапни големини')]")
        ul = div.find_element(By.XPATH, "following-sibling::ul[1]")
        sizes = [li.text.strip() for li in ul.find_elements(By.TAG_NAME, "li") if li.text.strip()]
    except Exception:
        pass

    sizes_str = ", ".join(sizes)

    print(f"[{idx}/{len(product_urls)}] {name} → Sizes: {sizes_str or '—'}")

    data.append({
        "Name": name,
        "Price": price,
        "Sizes": sizes_str,
        "Image_URL": image_url,
        "Product_URL": href
    })

# Зачувување со паднас дејтафрејм во цсв формат и нивно принтање во формат со име и величини
df = pd.DataFrame(data)
df.to_csv("kosuli_ff_fashion.csv", index=False, encoding="utf-8")

print(f"✔ Scraped {len(df)} products. Saved to kosuli_ff_fashion.csv.")
driver.quit()

→ Found 65 products across 6 pages.
[1/65] FFR054 REGULAR FIT LONG SLEEVE YELLOW/BLUE → Sizes: XXL
[2/65] FFR053 REGULAR FIT LONG SLEEVE BLACK/COLOR PRINT → Sizes: 3XL, 4XL
[3/65] FFR052 REGULAR FIT LONG SLEEVE RED/BLUE PRINT → Sizes: M, L, XL, 3XL, 4XL
[4/65] FFR051 REGULAR FIT LONG SLEEVE BLUE/LINE → Sizes: 3XL
[5/65] FFR050 REGULAR FIT LONG SLEEVE BLUE/RED LINE → Sizes: —
[6/65] FFR049 REGULAR FIT LONG SLEEVE YELLOW/BLUE → Sizes: XXL
[7/65] FFR048 REGULAR FIT LONG SLEEVE BLUE/WHITE/ORANGE → Sizes: L
[8/65] FFR047 REGULAR FIT LONG SLEEVE WHITE/PRINT → Sizes: L, 3XL
[9/65] FFR046 REGULAR FIT LONG SLEEVE ORANGE/BLUE PRINT → Sizes: M, L, XL, XXL, 3XL
[10/65] FFR045 REGULAR FIT LONG SLEEVE BLUE/OXFORD → Sizes: M, L, XL, XXL, 3XL
[11/65] FFS003 SLIM FIT LONG SLEEVE BLUE PRINT → Sizes: XL, XXL
[12/65] #FFR044# REGULAR FIT LONG SLEEVE BLUE OXFORD → Sizes: XL, XXL
[13/65] #FFR043# REGULAR FIT LONG SLEEVE WHITE\BLUE OXFORD → Sizes: —
[14/65] #FFR042# REGULAR FIT LONG SLEEVE WHITE\BLUE OXFORD 